### Load and Prepare Data

These steps are explained in the "DraftKingsEDA" document.

In [59]:
import numpy as np
import pandas as pd
from keras import optimizers
from keras.layers import Dropout
from sklearn import metrics
from keras.layers.normalization import BatchNormalization
from keras.layers import Activation

In [47]:
# Load Data
data = pd.read_csv('/Users/sauce/Desktop/DraftKings/ready_data/DraftKingsCleaned.csv', header = None)

# rename columns
data = data.rename(columns={0: "season", 
                        1:'game_date',
                        2: 'player',
                        3: 'team',
                        4: 'opponent',
                        5: 'venue',
                        6: 'minutes',
                        7: 'usage_rate',
                        8: 'rest',
                        9: 'avg_threes',
                        10: 'avg_reb',
                        11: 'avg_ast',
                        12: 'avg_stl',
                        13: 'avg_blk',
                        14: 'avg_tov',
                        15: 'avg_pts',
                        16: 'avg_points_vs_opp',
                        17: 'team_pace',
                        18: 'team_ast',
                        19: 'team_tov',
                        20: 'team_reb_rate',
                        21: 'team_offeff',
                        22: 'team_defeff',
                        23: 'opp_pace',
                        24: 'opp_ast',
                        25: 'opp_tov',
                        26: 'opp_reb_rate',
                        27: 'opp_offeff',
                        28: 'opp_defeff',
                        29: 'opp_pos_avg',
                        30: 'salary',
                        31: 'fantasy_points',
                       })

# Avg 10 data
data["game_date"] = pd.to_datetime(data.game_date)
data.set_index('game_date', inplace=True)
data.sort_index(inplace=True)
df_rolling = data.groupby(['player']).rolling(10).mean().rename(columns={'season':'season1', 'player':'player1'}).reset_index()
data = data.reset_index()
df_rolling = df_rolling.drop(columns=['player'])
df_rolling = df_rolling.rename(columns = {'season1': 'season', 'player1': 'player'})
data = pd.merge(data, df_rolling, on=['player', 'season', 'game_date'], left_index= True , suffixes=['', '_AVG10'])

# Avg 3 data
data["game_date"] = pd.to_datetime(data.game_date)
data.set_index('game_date', inplace=True)
data.sort_index(inplace=True)
df_rolling = data.groupby(['player']).rolling(3).mean().rename(columns={'season':'season1', 'player':'player1'}).reset_index()
data = data.reset_index()
df_rolling = df_rolling.drop(columns=['player'])
df_rolling = df_rolling.rename(columns = {'season1': 'season', 'player1': 'player'})
df = pd.merge(data, df_rolling, on=['player', 'season', 'game_date'], left_index= True , suffixes=['', '_AVG3'])

# Drop useless features
df = df.drop(columns=['team_AVG10','opponent_AVG10','venue_AVG10','rest_AVG10',
                      'avg_threes_AVG10','avg_reb_AVG10','avg_ast_AVG10','avg_stl_AVG10',
                      'avg_blk_AVG10','avg_tov_AVG10','avg_pts_AVG10','avg_points_vs_opp_AVG10',
                      'team_pace_AVG10','team_ast_AVG10','team_tov_AVG10','team_reb_rate_AVG10',
                      'team_defeff_AVG10','opp_pace_AVG10','opp_ast_AVG10','opp_tov_AVG10',
                      'opp_reb_rate_AVG10','opp_offeff_AVG10','salary_AVG10',
                      'team_AVG3', 'opponent_AVG3', 'venue_AVG3','rest_AVG3',
                      'avg_threes_AVG3', 'avg_reb_AVG3','avg_ast_AVG3', 'avg_stl_AVG3', 
                      'avg_blk_AVG3', 'avg_tov_AVG3','avg_pts_AVG3', 
                      'avg_points_vs_opp_AVG3', 'team_pace_AVG3','team_ast_AVG3', 
                      'team_tov_AVG3', 'team_reb_rate_AVG3','team_defeff_AVG3', 
                      'opp_pace_AVG3','opp_ast_AVG3', 'opp_tov_AVG3', 'opp_reb_rate_AVG3',
                      'opp_offeff_AVG3', 'opp_pos_avg_AVG3','salary_AVG3',
                      'team_AVG10_AVG3','opponent_AVG10_AVG3', 'venue_AVG10_AVG3', 
                      'minutes_AVG10_AVG3','usage_rate_AVG10_AVG3', 'rest_AVG10_AVG3',
                      'avg_threes_AVG10_AVG3', 'avg_reb_AVG10_AVG3',
                      'avg_ast_AVG10_AVG3', 'avg_stl_AVG10_AVG3', 'avg_blk_AVG10_AVG3',
                      'avg_tov_AVG10_AVG3', 'avg_pts_AVG10_AVG3',
                      'avg_points_vs_opp_AVG10_AVG3', 'team_pace_AVG10_AVG3',
                      'team_ast_AVG10_AVG3', 'team_tov_AVG10_AVG3',
                      'team_reb_rate_AVG10_AVG3', 'team_offeff_AVG10_AVG3',
                      'team_defeff_AVG10_AVG3', 'opp_pace_AVG10_AVG3',
                      'opp_ast_AVG10_AVG3', 'opp_tov_AVG10_AVG3',
                      'opp_reb_rate_AVG10_AVG3', 'opp_offeff_AVG10_AVG3',
                      'opp_defeff_AVG10_AVG3', 'opp_pos_avg_AVG10_AVG3',
                      'salary_AVG10_AVG3', 'fantasy_points_AVG10_AVG3'])

# Fill NAs
df['fantasy_points_AVG3'] = df['fantasy_points_AVG3'].fillna(df.groupby(['player', 'season'])['fantasy_points'].transform('mean'))
df['opp_defeff_AVG3'] = df['opp_defeff_AVG3'].fillna(df.groupby(['opponent', 'season'])['opp_defeff'].transform('mean'))
df['team_offeff_AVG3'] = df['team_offeff_AVG3'].fillna(df.groupby(['team', 'season'])['team_offeff'].transform('mean'))
df['usage_rate_AVG3'] = df['usage_rate_AVG3'].fillna(df.groupby(['player', 'season'])['usage_rate'].transform('mean'))
df['fantasy_points_AVG10'] = df['fantasy_points_AVG10'].fillna(df.groupby(['player', 'season'])['fantasy_points'].transform('mean'))
df['minutes_AVG3'] = df['minutes_AVG3'].fillna(df.groupby(['player', 'season'])['minutes'].transform('mean'))
df['opp_pos_avg_AVG10'] = df['opp_pos_avg_AVG10'].fillna(df.groupby(['opponent', 'season'])['opp_pos_avg'].transform('mean'))
df['opp_defeff_AVG10'] = df['opp_defeff_AVG10'].fillna(df.groupby(['opponent', 'season'])['opp_defeff'].transform('mean'))
df['team_offeff_AVG10'] = df['team_offeff_AVG10'].fillna(df.groupby(['team', 'season'])['team_offeff'].transform('mean'))
df['usage_rate_AVG10'] = df['usage_rate_AVG10'].fillna(df.groupby(['player', 'season'])['usage_rate'].transform('mean'))
df['minutes_AVG10'] = df['minutes_AVG10'].fillna(df.groupby(['player', 'season'])['minutes'].transform('mean'))

# Drop non predictive columns 
df2 = df.drop(columns=['game_date', 'season', 'player', 'team', 'opponent',
                     'minutes', 'usage_rate', 'salary'])
# Encode Dummies
df2 = pd.get_dummies(df2, columns=['venue', 'rest'], drop_first=True)

# Grab Target Variable and remove it from data.
y = df2['fantasy_points']
X = df2.drop(columns = ['fantasy_points'])

# Split data into train and test sets
from sklearn.cross_validation import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

# Scale data
from sklearn.preprocessing import StandardScaler
sc_X = StandardScaler()
X_train = sc_X.fit_transform(X_train)
X_test = sc_X.transform(X_test)

### ANN Regression

Help from https://machinelearningmastery.com/regression-tutorial-keras-deep-learning-library-python/

In [16]:
# Load Libraries
import numpy
import pandas
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

In [17]:
print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

(36460, 41) (36460,) (9116, 41) (9116,)


### Baseline Model

In [18]:
# Set seed so each model is evaluated similarly
seed = 7
numpy.random.seed(seed)

In [21]:
# Create Baseline Model
def baseline_model():
    model = Sequential()
    model.add(Dense(41, input_dim=41, kernel_initializer='normal', activation='relu'))
    model.add(Dense(1, kernel_initializer='normal'))
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model

In [22]:
# Standardize Data with Pipeline and evaluate model
numpy.random.seed(seed)
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasRegressor(build_fn=baseline_model, epochs=50, batch_size=5, verbose=0)))
pipeline = Pipeline(estimators)
kfold = KFold(n_splits=10, random_state=seed)
results = cross_val_score(pipeline, X, y, cv=kfold)
print("baseline: %.2f (%.2f) MSE" % (results.mean(), results.std()))

baseline: -34.78 (1.37) MSE


## Deeper Model

In [23]:
def deep_model():
    model = Sequential()
    model.add(Dense(41, input_dim=41, kernel_initializer='normal', activation='relu'))
    model.add(Dense(20, kernel_initializer='normal', activation='relu')) 
    model.add(Dense(10, kernel_initializer='normal', activation='relu'))    
    model.add(Dense(1, kernel_initializer='normal'))
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model

In [24]:
numpy.random.seed(seed)
estimators2 = []
estimators2.append(('standardize', StandardScaler()))
estimators2.append(('mlp', KerasRegressor(build_fn=deep_model, epochs=50, batch_size=5, verbose=0)))
pipeline = Pipeline(estimators2)
kfold = KFold(n_splits=10, random_state=seed)
results = cross_val_score(pipeline, X, y, cv=kfold)
print("deep: %.2f (%.2f) MSE" % (results.mean(), results.std()))

deep: -35.60 (1.88) MSE


### Try More Models

In [69]:
baseline_model = Sequential()
baseline_model.add(Dense(41, input_dim=41, kernel_initializer='normal', activation='relu'))

baseline_model.add(Dense(1, kernel_initializer='normal'))
baseline_model.compile(loss='mean_squared_error', optimizer='adam')

In [70]:
baseline_model = Sequential()
baseline_model.add(Dense(41, input_dim=41, kernel_initializer='normal', activation='relu'))
baseline_model.add(Dense(1, kernel_initializer='normal'))
baseline_model.compile(loss='mean_squared_error', optimizer='adam', metrics = ['accuracy'])

bL = baseline_model.fit(X_train, y_train,
                         batch_size = 50,
                         epochs = 50,
                         verbose = 2,
                         validation_data=(X_test, y_test))

Train on 36460 samples, validate on 9116 samples
Epoch 1/50
 - 3s - loss: 192.2204 - acc: 0.0119 - val_loss: 61.2917 - val_acc: 0.0131
Epoch 2/50
 - 2s - loss: 48.8471 - acc: 0.0155 - val_loss: 40.7190 - val_acc: 0.0194
Epoch 3/50
 - 1s - loss: 39.5980 - acc: 0.0198 - val_loss: 36.8743 - val_acc: 0.0224
Epoch 4/50
 - 1s - loss: 37.4609 - acc: 0.0211 - val_loss: 35.5995 - val_acc: 0.0236
Epoch 5/50
 - 1s - loss: 36.4488 - acc: 0.0213 - val_loss: 35.1011 - val_acc: 0.0219
Epoch 6/50
 - 2s - loss: 35.9034 - acc: 0.0216 - val_loss: 35.0832 - val_acc: 0.0225
Epoch 7/50
 - 1s - loss: 35.6087 - acc: 0.0227 - val_loss: 35.1589 - val_acc: 0.0217
Epoch 8/50
 - 2s - loss: 35.4729 - acc: 0.0225 - val_loss: 34.6417 - val_acc: 0.0252
Epoch 9/50
 - 2s - loss: 35.2697 - acc: 0.0228 - val_loss: 34.4484 - val_acc: 0.0244
Epoch 10/50
 - 2s - loss: 35.1616 - acc: 0.0226 - val_loss: 34.4755 - val_acc: 0.0253
Epoch 11/50
 - 1s - loss: 35.1493 - acc: 0.0234 - val_loss: 34.7813 - val_acc: 0.0253
Epoch 12/50
 

In [71]:
bl_pred = baseline_model.predict(X_test)
print('BL Mean Squared Error:', metrics.mean_squared_error(y_test, bl_pred))

BL Mean Squared Error: 34.45671784167841


In [48]:
model = Sequential()
model.add(Dense(41, input_dim=41, activation='relu'))
model.add(Dense(20, activation='relu')) 
model.add(Dense(20, activation='relu')) 
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer = 'adam', metrics = ['accuracy'])

mod = model.fit(X_train, y_train,
                         batch_size = 10,
                         epochs = 50,
                         verbose = 2,
                         validation_data=(X_test, y_test))

Train on 36460 samples, validate on 9116 samples
Epoch 1/50
 - 9s - loss: 54.5563 - acc: 0.0183 - val_loss: 36.9575 - val_acc: 0.0173
Epoch 2/50
 - 7s - loss: 37.1263 - acc: 0.0202 - val_loss: 35.6939 - val_acc: 0.0210
Epoch 3/50
 - 7s - loss: 36.4677 - acc: 0.0207 - val_loss: 34.7379 - val_acc: 0.0217
Epoch 4/50
 - 7s - loss: 36.0920 - acc: 0.0222 - val_loss: 35.5665 - val_acc: 0.0233
Epoch 5/50
 - 7s - loss: 35.9208 - acc: 0.0217 - val_loss: 34.5723 - val_acc: 0.0234
Epoch 6/50
 - 7s - loss: 35.7129 - acc: 0.0224 - val_loss: 34.5156 - val_acc: 0.0236
Epoch 7/50
 - 7s - loss: 35.6339 - acc: 0.0222 - val_loss: 34.6699 - val_acc: 0.0240
Epoch 8/50
 - 8s - loss: 35.5326 - acc: 0.0215 - val_loss: 34.5685 - val_acc: 0.0215
Epoch 9/50
 - 7s - loss: 35.3429 - acc: 0.0207 - val_loss: 36.5502 - val_acc: 0.0230
Epoch 10/50
 - 8s - loss: 35.1216 - acc: 0.0214 - val_loss: 34.4428 - val_acc: 0.0238
Epoch 11/50
 - 8s - loss: 34.9393 - acc: 0.0231 - val_loss: 34.9854 - val_acc: 0.0236
Epoch 12/50
 -

In [52]:
mod_pred = model.predict(X_test)
print('Mod Mean Squared Error:', metrics.mean_squared_error(y_test, mod_pred))

Mod Mean Squared Error: 34.92576497868474


In [49]:
from keras import optimizers
from keras.layers import Dropout

optimizer=optimizers.SGD(lr=1e-4)


model_drop = Sequential()
model_drop.add(Dense(41, input_dim=41, kernel_initializer='normal', activation='relu'))
model_drop.add(Dropout(0.2))
model_drop.add(Dense(20, kernel_initializer='normal', activation='relu')) 
model_drop.add(Dense(1, kernel_initializer='normal'))
model_drop.compile(loss='mean_squared_error', optimizer = optimizer)

mod_drop = model_drop.fit(X_train, y_train,
                         batch_size = 25,
                         epochs = 50,
                         verbose = 2,
                         validation_data=(X_test, y_test))

Train on 36460 samples, validate on 9116 samples
Epoch 1/50
 - 4s - loss: 223.0513 - val_loss: 43.5507
Epoch 2/50
 - 3s - loss: 44.9450 - val_loss: 37.2871
Epoch 3/50
 - 3s - loss: 42.0394 - val_loss: 35.7084
Epoch 4/50
 - 3s - loss: 40.2149 - val_loss: 34.7238
Epoch 5/50
 - 3s - loss: 39.7726 - val_loss: 34.5061
Epoch 6/50
 - 3s - loss: 39.1815 - val_loss: 34.3274
Epoch 7/50
 - 3s - loss: 39.0411 - val_loss: 34.3695
Epoch 8/50
 - 3s - loss: 38.7401 - val_loss: 34.3557
Epoch 9/50
 - 4s - loss: 38.5094 - val_loss: 34.4166
Epoch 10/50
 - 4s - loss: 38.5869 - val_loss: 34.2529
Epoch 11/50
 - 3s - loss: 38.2591 - val_loss: 34.3956
Epoch 12/50
 - 3s - loss: 38.0791 - val_loss: 34.2066
Epoch 13/50
 - 3s - loss: 38.4785 - val_loss: 34.1686
Epoch 14/50
 - 3s - loss: 38.3083 - val_loss: 34.1332
Epoch 15/50
 - 3s - loss: 38.3377 - val_loss: 34.2997
Epoch 16/50
 - 3s - loss: 38.1374 - val_loss: 34.3937
Epoch 17/50
 - 4s - loss: 38.1713 - val_loss: 34.5177
Epoch 18/50
 - 3s - loss: 37.9884 - val_l

In [53]:
drop_pred = model_drop.predict(X_test)
print('Drop Mean Squared Error:', metrics.mean_squared_error(y_test, drop_pred))

Drop Mean Squared Error: 34.258480332341854


In [61]:
from keras import optimizers
from keras.layers import Dropout

optimizer=optimizers.Adam(lr=1e-4)

model_drop = Sequential()
model_drop.add(Dense(41, input_dim=41, kernel_initializer='normal'))
model_drop.add(BatchNormalization())
model_drop.add(Activation('relu'))
model_drop.add(Dropout(0.2))
model_drop.add(Dense(20, kernel_initializer='normal')) 
model_drop.add(BatchNormalization())
model_drop.add(Activation('relu'))
model_drop.add(Dense(1, kernel_initializer='normal'))
model_drop.compile(loss='mean_squared_error', optimizer = optimizer)

mod_drop = model_drop.fit(X_train, y_train,
                         batch_size = 25,
                         epochs = 50,
                         verbose = 2,
                         validation_data=(X_test, y_test))

Train on 36460 samples, validate on 9116 samples
Epoch 1/50
 - 6s - loss: 581.6032 - val_loss: 512.8105
Epoch 2/50
 - 4s - loss: 437.4663 - val_loss: 354.4827
Epoch 3/50
 - 4s - loss: 280.5139 - val_loss: 214.4454
Epoch 4/50
 - 4s - loss: 164.9904 - val_loss: 115.5466
Epoch 5/50
 - 4s - loss: 99.8152 - val_loss: 78.7052
Epoch 6/50
 - 4s - loss: 76.5000 - val_loss: 60.1273
Epoch 7/50
 - 4s - loss: 67.1459 - val_loss: 46.6267
Epoch 8/50
 - 4s - loss: 55.9036 - val_loss: 41.9886
Epoch 9/50
 - 5s - loss: 49.8947 - val_loss: 38.0543
Epoch 10/50
 - 4s - loss: 49.2338 - val_loss: 37.4050
Epoch 11/50
 - 4s - loss: 47.7352 - val_loss: 37.1262
Epoch 12/50
 - 4s - loss: 47.2884 - val_loss: 38.5574
Epoch 13/50
 - 4s - loss: 46.9246 - val_loss: 36.1673
Epoch 14/50
 - 4s - loss: 46.8139 - val_loss: 36.8396
Epoch 15/50
 - 4s - loss: 46.7557 - val_loss: 36.1361
Epoch 16/50
 - 4s - loss: 46.3714 - val_loss: 36.1514
Epoch 17/50
 - 4s - loss: 46.1592 - val_loss: 36.5205
Epoch 18/50
 - 5s - loss: 46.2977 

In [66]:
batch_pred = model_drop.predict(X_test)
print('batch Mean Squared Error:', metrics.mean_squared_error(y_test, batch_pred))

batch Mean Squared Error: 35.63643866212873


In [63]:
from keras import optimizers
from keras.layers import Dropout

optimizer=optimizers.Adam(lr=1e-4)

model_drop = Sequential()
model_drop.add(Dense(100, input_dim=41, kernel_initializer='normal'))
model_drop.add(BatchNormalization())
model_drop.add(Activation('relu'))
model_drop.add(Dropout(0.2))
model_drop.add(Dense(40, input_dim=41, kernel_initializer='normal'))
model_drop.add(BatchNormalization())
model_drop.add(Activation('relu'))
model_drop.add(Dropout(0.2))
# model_drop.add(Dense(20, kernel_initializer='normal')) 
# model_drop.add(BatchNormalization())
# model_drop.add(Activation('relu'))
model_drop.add(Dense(1, kernel_initializer='normal'))
model_drop.compile(loss='mean_squared_error', optimizer = optimizer)

mod_drop = model_drop.fit(X_train, y_train,
                         batch_size = 25,
                         epochs = 50,
                         verbose = 2,
                         validation_data=(X_test, y_test))

Train on 36460 samples, validate on 9116 samples
Epoch 1/50
 - 6s - loss: 532.9647 - val_loss: 422.9229
Epoch 2/50
 - 5s - loss: 306.3374 - val_loss: 204.9261
Epoch 3/50
 - 5s - loss: 148.0012 - val_loss: 97.8653
Epoch 4/50
 - 6s - loss: 84.7241 - val_loss: 52.9068
Epoch 5/50
 - 5s - loss: 59.2355 - val_loss: 41.2527
Epoch 6/50
 - 5s - loss: 53.2597 - val_loss: 40.2472
Epoch 7/50
 - 5s - loss: 52.2258 - val_loss: 37.5299
Epoch 8/50
 - 6s - loss: 50.9376 - val_loss: 37.8071
Epoch 9/50
 - 5s - loss: 51.2663 - val_loss: 36.5861
Epoch 10/50
 - 5s - loss: 50.7393 - val_loss: 36.7313
Epoch 11/50
 - 5s - loss: 50.2247 - val_loss: 37.7761
Epoch 12/50
 - 5s - loss: 49.7176 - val_loss: 36.5645
Epoch 13/50
 - 5s - loss: 50.1932 - val_loss: 37.9390
Epoch 14/50
 - 5s - loss: 49.6536 - val_loss: 36.0982
Epoch 15/50
 - 5s - loss: 49.1669 - val_loss: 36.6192
Epoch 16/50
 - 5s - loss: 49.8771 - val_loss: 36.0788
Epoch 17/50
 - 5s - loss: 49.7778 - val_loss: 36.1335
Epoch 18/50
 - 5s - loss: 48.2481 - v

In [65]:
c_pred = model_drop.predict(X_test)
print('c Mean Squared Error:', metrics.mean_squared_error(y_test, c_pred))

c Mean Squared Error: 35.63643866212873


In [67]:
optimizer=optimizers.Adam(lr=1e-4)

model_drop = Sequential()
model_drop.add(Dense(50, input_dim=41))
model_drop.add(BatchNormalization())
model_drop.add(Activation('relu'))
model_drop.add(Dense(1))
model_drop.compile(loss='mean_squared_error', optimizer = optimizer)

mod_drop = model_drop.fit(X_train, y_train,
                         batch_size = 25,
                         epochs = 50,
                         verbose = 2,
                         validation_data=(X_test, y_test))

Train on 36460 samples, validate on 9116 samples
Epoch 1/50
 - 5s - loss: 521.5445 - val_loss: 393.0840
Epoch 2/50
 - 3s - loss: 306.0540 - val_loss: 222.0777
Epoch 3/50
 - 4s - loss: 154.0185 - val_loss: 98.2885
Epoch 4/50
 - 4s - loss: 72.2389 - val_loss: 50.3678
Epoch 5/50
 - 3s - loss: 50.6188 - val_loss: 43.9389
Epoch 6/50
 - 4s - loss: 48.0841 - val_loss: 42.0035
Epoch 7/50
 - 3s - loss: 47.4199 - val_loss: 40.8737
Epoch 8/50
 - 4s - loss: 47.5700 - val_loss: 39.6313
Epoch 9/50
 - 4s - loss: 46.1830 - val_loss: 39.0208
Epoch 10/50
 - 4s - loss: 46.2360 - val_loss: 38.4373
Epoch 11/50
 - 4s - loss: 45.2450 - val_loss: 37.8036
Epoch 12/50
 - 4s - loss: 45.6086 - val_loss: 37.3550
Epoch 13/50
 - 4s - loss: 45.1765 - val_loss: 37.0252
Epoch 14/50
 - 6s - loss: 45.0700 - val_loss: 36.7932
Epoch 15/50
 - 4s - loss: 45.2938 - val_loss: 36.5533
Epoch 16/50
 - 4s - loss: 44.2918 - val_loss: 36.2307
Epoch 17/50
 - 4s - loss: 44.1215 - val_loss: 36.0570
Epoch 18/50
 - 5s - loss: 43.9777 - v

In [68]:
d_pred = model_drop.predict(X_test)
print('d Mean Squared Error:', metrics.mean_squared_error(y_test, d_pred))

d Mean Squared Error: 34.69197885755987


In [ ]:
model_two = Sequential()
model_two.add(Dense(41, input_dim=41, kernel_initializer='normal', activation='relu'))
model_two.add(Dense(20, kernel_initializer='normal', activation='relu')) 
model_two.add(Dense(1, kernel_initializer='normal'))
model_two.compile(loss='mean_squared_error', optimizer = 'adam')

monitor = EarlyStopping